In [58]:
import ipywidgets as widgets
from ipywidgets import Box
from IPython.display import display, clear_output
from enum import Enum
from collections import namedtuple
import requests
import json

In [59]:
GeodexWebServiceAction = namedtuple('GeodexWebServiceAction', ['value', 'web_service_type', 'url_suffix'])

class GeodexWebServiceActions(Enum):
    TEXTINDEX_SEARCH = GeodexWebServiceAction(0, "GET", "textindex/search")
    TEXTINDEX_SEARCHSET = GeodexWebServiceAction(0, "GET", "textindex/searchset")
    SPATIAL_SEARCH_OBJECT = GeodexWebServiceAction(0, "GET", "spatial/search/object") 
    SPATIAL_SEARCH_RESOURCE = GeodexWebServiceAction(0, "GET", "spatial/search/resource")
    SPATIAL_SEARCH_RESOURCESET = GeodexWebServiceAction(0, "POST", "spatial/search/resourceset") 
    TYPEAHEAD_PROVIDERS = GeodexWebServiceAction(0, "GET", "typeahead/providers")
    GRAPH_RESDETAILS = GeodexWebServiceAction(0, "GET", "graph/resdetails")
    GRAPH_RESSETDETAILS = GeodexWebServiceAction(0, "POST", "graph/ressetdetails")
    GRAPH_RESSETPEOPLE = GeodexWebServiceAction(0, "POST", "graph/ressetpeople")

    def get_web_service_type(self):
        return self.value.web_service_type

    def get_url_suffix(self):
        return self.value.url_suffix

In [60]:
class GeodexWebServiceClient(object):
    
    def perform_web_service_call(self, geodex_web_service_action, data_dict, update_function):

        #Set core domain name for request URL
        domainURL = "http://geodex.org/api/v1/"

        #Create request URL
        requestURL = domainURL

        #Add the suffix for the call
        requestURL += geodex_web_service_action.get_url_suffix()

        #Get the type of request URL: GET or POST 
        web_service_type = geodex_web_service_action.get_web_service_type()
        
        #Send the request to the server
        if web_service_type=="POST":
            r = requests.get(requestURL, params=data_dict)
        elif web_service_type=="GET":
            r = requests.get(requestURL, params=data_dict)
        
        #Check status code and process data
        if r.status_code == requests.codes.ok:
            update_function(json.loads(json.dumps(r.json())))
        

In [61]:
class MainData(object):
    
    def __init__(self):
        self._providers = []
        self._selected_providers = []
        self._text_results = []
        
    def set_text_search_selector(self, text_search_selector):
        self._text_search_selector = text_search_selector
        
    def process_providers(self, data):
        self._providers = []
        for row in data:
            index = row["index"]
            name = row["name"]
            description = row["description"]
            provider = Provider(index, name, description)
            self._providers.append(provider)
    
    def process_text_results(self, data):
        self._text_results = []
        for row in data:
            print(row)
            position = row["position"]
            score = row["score"]
            url = row["URL"]
            text_result = TextResult(position, score, url)
            self._text_results.append(text_result)
    
    def process_selected_provider_text_results(self, data):
        self._selected_providers = []
        for row in data:
            index = row["index"]
            high_score = row["highscore"]
            text_result_set = row["or"]
            provider = self._get_provider_by_index(index)
            provider_is_selected = self._is_provider_selected(provider.get_name())
            if provider_is_selected:
                self._selected_providers.append(provider)
                provider.set_high_score(high_score)
                provider.process_text_results(text_result_set)
    
    def _get_provider_by_index(self, index):
        for provider in self._providers:
            if provider.get_index() == index:
                return provider
    
    def _is_provider_selected(self, name):
        box_array = self._text_search_selector.get_box_array()
        for box in box_array:
            if box.description == name and box.value == True:
                return True
        return False
    
    def get_providers(self):
        return self._providers
    
    def get_text_results(self):
        return self._text_results
    
    def get_selected_providers(self):
        return self._selected_providers
        

In [62]:
class Provider(object):
    
    def __init__(self, index, name, description):
        self._index = index
        self._name = name
        self._description = description
        self._text_results = []
     
    def process_text_results(self, data):
        self._text_results = []
        for row in data:
            position = row["position"]
            score = row["score"]
            url = row["URL"]
            text_result = TextResult(position, score, url)
            self._text_results.append(text_result)

    def get_index(self):
        return self._index
    
    def get_name(self):
        return self._name
    
    def get_description(self):
        return self._description
    
    def get_text_results(self):
        return self._text_results
    
    def set_high_score(self, high_score):
        self._high_score = high_score
        
    def get_high_score(self):
        return self._high_score
    
    def get_text_results(self):
        return self._text_results

In [63]:
class TextResult(object):
    
    def __init__(self, position, score, url):
        self._position = position
        self._score = score
        self._url = url
        
    def get_position(self):
        return self._position
    
    def get_score(self):
        return self._score
    
    def get_url(self):
        return self._url

In [64]:
class TextSearchSelector(object):

    def __init__(self, geodex_web_service_client, main_data, text_search_viewer, text_searchset_viewer):
        self._geodex_web_service_client = geodex_web_service_client
        self._main_data = main_data
        self._text_search_viewer = text_search_viewer
        self._text_searchset_viewer = text_searchset_viewer
        self._geodex_web_service_client.perform_web_service_call(GeodexWebServiceActions.TYPEAHEAD_PROVIDERS, {}, self.update_after_typeahead_providers)
        self._layout_ui()
        self.show_ui()
    
    def show_ui(self):
        display(self._container)
    
    def hide_ui(self):
        clear_output()
    
    def update_after_typeahead_providers(self, data):
        main_data.process_providers(data)
    
    def get_text_search_value(self):
        return self._text_box.value
    
    def get_box_array(self):
        return self._box_array
    
    def _on_submit_button_clicked(self, button):

        if self._radio_buttons.value == "Get Top Results For Each Selected Provider":
            q = self._text_box.value
            s = '0'
            n = str(self._max_num_results_box.value)
            data_dict = {'q':q,'s':s,'n': n}
            self._geodex_web_service_client.perform_web_service_call(GeodexWebServiceActions.TEXTINDEX_SEARCHSET, 
                                                                     data_dict, 
                                                                     self.update_after_textindex_searchset)
        else:
            q = self._text_box.value
            s = '0'
            n = self._max_num_results_box.value
            i = self._get_selected_provider_index_list()
            data_dict = {'q': q, 's': s, 'n': n, 'i': i}
            self._geodex_web_service_client.perform_web_service_call(GeodexWebServiceActions.TEXTINDEX_SEARCH, 
                                                                     data_dict, 
                                                                     self.update_after_textindex_search)
    
    def update_after_textindex_searchset(self, data):
        self._main_data.process_selected_provider_text_results(data)
        self._text_search_viewer.hide_ui()
        self._text_searchset_viewer.initialize_ui()
        self._text_searchset_viewer.show_ui()
        
    def update_after_textindex_search(self, data):
        self._main_data.process_text_results(data)
        self._text_searchset_viewer.hide_ui()
        self._text_search_viewer.initialize_ui()
        self._text_search_viewer.show_ui()
    
    def _get_selected_provider_index_list(self):
        index_list_str = ""
        for provider in main_data.get_providers():
            name = provider.get_name()
            if self._is_provider_checked(name):
                index = self._get_provider_index_by_name(name)
                index_list_str +=  index + ","
        return index_list_str[:-1]
    
    def _is_provider_checked(self, name):
        for box in self._box_array:
            if box.description == name:
                return box.value
    
    def _get_provider_index_by_name(self, name):
        for provider in main_data.get_providers():
            if provider.get_name() == name:
                return provider.get_index()
    
    def _layout_ui(self):
        
        #Labels
        self._text_label = widgets.Label(
            value = 'Enter Text Search Value: ', 
            layout = widgets.Layout(
                width = '25%', 
                margin = '5px 5px 5px 50px', 
            )
        )
        
        self._max_num_results_label = widgets.Label(
            value = 'Select Max Number of Top Results: ', 
            layout = widgets.Layout(
                width = '30%', 
                margin = '5px 5px 5px 5px', 
            )
        )
        
        #Textfields
        self._text_box = widgets.Text(
            value = 'Carbon',
            layout = widgets.Layout(
                width = '25%', 
                margin = '5px 50px 5px 0px'
            )
        )
        
        #Spinners
        self._max_num_results_box = widgets.BoundedIntText(
            value=10,
            min=10,
            max=250,
            step=10,
            description='',
            disabled=False, 
            layout = widgets.Layout(
                width = '20%', 
                margin = '5px 50px 5px 0px'
            )
        )
        
        #Buttons
        self._submit_button = widgets.Button(
            description = 'Submit Search to Geodex.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._submit_button.on_click(self._on_submit_button_clicked)
        
        #Check boxes
        self._box_array = []
        for provider in self._main_data.get_providers():
            box = widgets.Checkbox(
                value=True,
                description=provider.get_name(),
                disabled=False
            )
            self._box_array.append(box)
        
        #Radio Buttons
        self._radio_buttons = widgets.RadioButtons(
            options=['Get Top Results For Each Selected Provider', 'Get Top Results Across All Selected Providers'],
            value='Get Top Results For Each Selected Provider',
            description='',
            disabled=False
        )
        
        #Layout components
        self._top_container = widgets.HBox(
            [self._text_label, 
             self._text_box, 
             self._max_num_results_label, 
             self._max_num_results_box]
        )
        self._top_container.margin = '15px 0px 15px 0px'
        
        self._middle_container = Box()
        self._middle_container.layout.display = 'flex'
        self._middle_container.layout.flex_flow = 'row wrap'
        self._middle_container.layout.align_items = 'stretch'
        self._middle_container.layout.flex_wrap = 'wrap'
        self._middle_container.layout.margin = '15px 0px 15px 0px'
        self._middle_container.children = self._box_array
        
        self._bottom_container = widgets.HBox(
            [self._radio_buttons]
        )
        self._bottom_container.layout.justify_content = 'center'
        self._bottom_container.layout.margin = '15px 0px 15px 0px'
        
        self._container = widgets.VBox(
            [self._top_container, 
             self._middle_container, 
             self._bottom_container,
             self._submit_button]
        )

In [65]:
class TextSearchViewer(object):
    
    def __init__(self, geodex_web_service_client, main_data):
        self._geodex_web_service_client = geodex_web_service_client
        self._main_data = main_data
        self._layout_ui()
       
    def set_text_search_selector(self, text_search_selector):
        self._text_search_selector = text_search_selector
    
    def initialize_ui(self): 
        html = "<table>"
        for text_result in self._main_data.get_text_results():
            html += "<tr><td>"
            html += "<a href=\"" + text_result.get_url() + "\" target=\"_blank\">" + text_result.get_url() + "</a>"
            html += "</td></tr>"
            html += "<tr><td>" 
            html += "Position: " + str(text_result.get_position())
            html += "</td></tr>"
            html += "<tr><td>" 
            html += "Score: " + str(text_result.get_score())
            html += "</td></tr>"
        html += "</table>"
        self._results_list.value = html
    
    def _on_back_button_clicked(self, button):
        self.hide_ui()
        self._text_search_selector.show_ui()

    def _layout_ui(self):  
        
        #Buttons
        self._back_button = widgets.Button(
            description = 'Modify Text Search at Geodex.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._back_button.on_click(self._on_back_button_clicked)
        
        #List
        self._results_list = widgets.HTML(
            value=""
        )
        
        self._container = widgets.VBox(
            [self._results_list, 
             self._back_button]
        )
    
    def show_ui(self):
        display(self._container)
        
    def hide_ui(self):
        clear_output()
    

In [71]:
class TextSearchsetViewer(object):
    
    def __init__(self, geodex_web_service_client, main_data):
        self._geodex_web_service_client = geodex_web_service_client
        self._main_data = main_data
        self._layout_ui()
        
    def set_text_search_selector(self, text_search_selector):
        self._text_search_selector = text_search_selector
    
    def initialize_ui(self):
        options = []
        for provider in self._main_data.get_selected_providers():
            if len(provider.get_text_results()) > 0:
                options.append(provider.get_name())
        self._provider_list.options = options
        self._provider_list.value = options[0]
        
    def _on_back_button_clicked(self, button):
        self.hide_ui()
        self._text_search_selector.show_ui()
        
    def _on_provider_list_selected(self, select):
        self._update_results_list()
        
    def _get_provider_by_name(self, name):
        for provider in self._main_data.get_selected_providers():
            if provider.get_name() == name:
                return provider
        
    def _update_results_list(self):
            name = self._provider_list.value
            provider = self._get_provider_by_name(name)
            if provider is not None:
                html = "<table>"
                for text_result in provider.get_text_results():
                    html += "<tr><td>"
                    html += "<a href=\"" + text_result.get_url() + "\" target=\"_blank\">" + text_result.get_url() + "</a>"
                    html += "</td></tr>"
                    html += "<tr><td>" 
                    html += "Position: " + str(text_result.get_position())
                    html += "</td></tr>"
                    html += "<tr><td>" 
                    html += "Score: " + str(text_result.get_score())
                    html += "</td></tr>"
                html += "</table>"
                self._results_list.value = html
    
    def _layout_ui(self):  
        
        #Buttons
        self._back_button = widgets.Button(
            description = 'Modify Text Search at Geodex.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._back_button.on_click(self._on_back_button_clicked)
        
        #List
        self._provider_list = widgets.Select(
            options=['test'],
            value='test',
            description='',
            disabled=False
        )
        self._provider_list.observe(self._on_provider_list_selected)
        
        self._results_list = widgets.HTML(
            value=""
        )
        
        self._top_container = widgets.HBox(
            [self._provider_list, 
             self._results_list]
        )
        
        self._container = widgets.VBox(
            [self._top_container, 
             self._back_button]
        )
        
    def show_ui(self):
        display(self._container)
        
    def hide_ui(self):
        clear_output()


In [72]:
geodex_web_service_client = GeodexWebServiceClient()
main_data = MainData()
text_search_viewer = TextSearchViewer(geodex_web_service_client, main_data)
text_searchset_viewer = TextSearchsetViewer(geodex_web_service_client, main_data)
text_search_selector = TextSearchSelector(geodex_web_service_client, main_data, 
                                          text_search_viewer, text_searchset_viewer)

text_search_viewer.set_text_search_selector(text_search_selector)
text_searchset_viewer.set_text_search_selector(text_search_selector)
main_data.set_text_search_selector(text_search_selector)

A Jupyter Widget